In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Oct_dict_JSON.json', 'r') as json_file:
    link_day_minute_Oct_dict_JSON = json.load(json_file)

In [10]:
week_day_Oct_list = [1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 29, 30, 31]

In [11]:
link_day_minute_Oct_list = []
for link_idx in range(24):
    for day in week_day_Oct_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Oct_list.append(link_day_minute_Oct_dict_JSON[key] ['NT_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Oct_list)

66240

In [13]:
x = np.matrix(link_day_minute_Oct_list)
x = np.matrix.reshape(x, 24, 2760)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2760)

In [15]:
x[:,:2]

matrix([[ 3585.47279386,  3576.0851033 ],
        [ 3477.40930602,  3494.69857302],
        [ 4464.31122495,  4439.79209736],
        [ 4372.10239005,  4390.81014768],
        [ 4335.43999342,  4199.46306071],
        [ 4150.76760104,  4004.80678194],
        [ 3693.44131357,  3635.07355358],
        [ 3705.53382992,  3681.48036626],
        [ 5013.72120339,  4522.20047052],
        [ 4751.58429303,  4738.36840881],
        [ 3987.4511081 ,  4106.01736415],
        [ 3655.75935003,  3768.55578966],
        [ 3323.89838984,  3361.45839178],
        [ 3746.61357103,  3762.4352956 ],
        [ 3698.76066198,  3482.23607329],
        [ 2802.12369269,  2754.42225222],
        [ 3702.69706733,  3661.01543763],
        [ 3625.79851096,  3607.08659819],
        [ 3951.77655753,  3996.17568765],
        [ 3817.67556057,  3720.6900137 ],
        [ 3776.11532857,  3931.97464469],
        [ 3941.18120826,  3826.26936239],
        [ 3586.49827764,  3410.46402514],
        [ 3704.87886873,  3551.567

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [1e+00, 9e+02]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 25
 AA' NZ     : 3.000e+02
 Factor NZ  : 3.250e+02
 Factor Ops : 5.525e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.51119801e+07  0.00000000e+00  5.44e+03 1.24e+03  1.00e+06     0s
   1  -1.08069370e+07 -4.88222061e+06  1.65e+03 3.77e+02  3.62e+05     0s
   2  -7.41027158e+06 -8.70957707e+06  1.27e+02 2.90e+01  5.96e+04     0s
   3  -7.38868954e+06 -7.86405712e+06  1.27e-04 2.90e-05  8.4

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Oct_weekday_NT.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[3.412813074229544e-11,
 3504.793874501573,
 3.7461442605062415e-11,
 0.002017279197427762,
 1168.309055719352,
 1810.345522860911,
 1.1304700826444268e-08,
 5.354830559586359e-05,
 3.1220642301113415,
 3752.134175753437,
 0.005615928684767011,
 0.6724729271610927,
 280.3440344512054,
 0.15657272324530697,
 4022.2839284778925,
 1156.655143513876,
 343.87934137900214,
 1.5967468205902775,
 0.6584558030349873,
 4634.520380136688,
 1.3555069640462332,
 2110.3727179090547,
 1647.2472692213703,
 589.2306973816765,
 2424.0563151602105,
 1.3720127843543972,
 0.00015401403766151866,
 3584.2718126565987,
 2.6577525103088426,
 2.3039367701590905,
 26.00684881701116,
 553.5382874137314,
 3003.5267162439404,
 0.00015245974023064108,
 0.00015255859990282377,
 1.399957794777336,
 0.7516475740490307,
 146.81358851594234,
 2750.7272818089414,
 1.0052458856589979,
 271.2674065439755,
 595.5090999263607,
 1699.3210709751481,
 1311.8207357150393,
 851.4425235830047,
 2.9667274210852693e-06,
 2.9305756780